In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
import numpy as np
import cv2
import os

# Define the input shape
input_shape = (224, 224, 3)  # Using a smaller input size to reduce memory footprint

# Define the maximum number of objects per image
num_boxes = 5

# Create the CNN model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))



# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print the model summary
model.summary()

# Load and preprocess the data
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0  # Normalize pixel values
    return image



def preprocess_labels(label_path):
    with open(label_path, 'r') as f:
        label_str = f.read().strip()

    labels = []
    for label in label_str.split('\n'):
        if label.strip():
            label_data = label.split()
            class_id = int(label_data[0])
            x = float(label_data[1])
            y = float(label_data[2])
            w = float(label_data[3])
            h = float(label_data[4])
            labels.append([class_id, x, y, w, h])

    # Pad or truncate labels to have exactly num_boxes entries
    labels = labels[:num_boxes]  # Truncate if more than num_boxes objects
    labels.extend([[0, 0, 0, 0, 0]] * (num_boxes - len(labels)))  # Pad with zeros if fewer than num_boxes objects

    # Convert to numpy array
    labels = np.array(labels)

    return labels




def load_data(image_dir, label_dir):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')])
    label_paths = sorted([os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')])

    images = np.array([preprocess_image(path) for path in image_paths])
    labels = np.array([preprocess_labels(path) for path in label_paths])

    return images, labels

train_image_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\train\non_tilted_images'
train_label_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\train\non_tilted_labels'
val_image_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\valid\images'
val_label_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\valid\labels'

train_images, train_labels = load_data(train_image_dir, train_label_dir)
val_images, val_labels = load_data(val_image_dir, val_label_dir)

# Define the maximum number of objects per image
num_boxes = 5

# Determine the maximum number of elements in the labels
# Determine the maximum number of elements in the labels
max_label_elements = max(len(label) for label in train_labels)

# Update the output size based on the maximum number of elements in the labels
output_size = max_label_elements * num_boxes

# Output layer for object detection
model.add(Dense(output_size, activation='linear'))
model.add(Reshape((num_boxes, max_label_elements)))  # Reshape the output to [num_boxes, max_label_elements]

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 64)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,177,216 (42.64 MB)

 Trainable params: 11,177,216 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.3108 - loss: 1.9739 - val_accuracy: 0.8534 - val_loss: 0.3864
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.6670 - loss: 0.5000 - val_accuracy: 0.8534 - val_loss: 0.2928
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.7898 - loss: 0.3867 - val_accuracy: 0.8534 - val_loss: 0.3022
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 119s 2s/step - accuracy: 0.8181 - loss: 0.3318 - val_accuracy: 0.8534 - val_loss: 0.1992
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.8378 - loss: 0.2445 - val_accuracy: 0.8534 - val_loss: 0.1543
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.8483 - loss: 0.2209 - val_accuracy: 0.8534 - val_loss: 0.1574
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.8421 - loss: 0.2029 - val_accuracy: 0.8653 - val_loss: 0.1269
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.8513 - loss: 0.1899 - val_accuracy: 0.8707 - v

In [42]:
from tensorflow.keras.models import save_model

# Assuming `model` is your Keras model
save_model(model, 'luk_cnn_model.keras')  # Save the model in the native Keras format


In [9]:
# Load and preprocess test data
test_image_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\test\images'
test_label_dir = r'C:\Users\LENOVO L460\Downloads\ipbl\Capstone-V-Leaukemia Detection\test\labels'

test_images, test_labels = load_data(test_image_dir, test_label_dir)

# Evaluate the model on test data
loss, accuracy = model.evaluate(test_images, test_labels)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 311ms/step - accuracy: 0.7171 - loss: 0.0978
Test Loss: 0.1172095462679863
Test Accuracy: 0.8814102411270142


In [10]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_iou(box1, box2):
    # Calculate intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculate area of intersection rectangle
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Calculate area of both boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Calculate IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    return iou

# Predict on test data
predictions = model.predict(test_images)

# Flatten the predictions and ground truth labels for easier calculation
predictions_flat = predictions.reshape(-1, num_boxes * max_label_elements)
test_labels_flat = test_labels.reshape(-1, num_boxes * max_label_elements)

# Convert predictions to binary values based on a threshold (e.g., 0.5)
threshold = 0.5
binary_predictions = (predictions_flat > threshold).astype(int)
binary_test_labels = (test_labels_flat > threshold).astype(int)

# Calculate precision, recall, and F1-score
precision = precision_score(binary_test_labels, binary_predictions, average='weighted')
recall = recall_score(binary_test_labels, binary_predictions, average='weighted')
f1 = f1_score(binary_test_labels, binary_predictions, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Calculate IoU for each prediction
ious = []
for i in range(len(predictions)):
    for j in range(num_boxes):
        pred_box = predictions[i][j]
        true_box = test_labels[i][j][1:5]  # Extracting the bounding box coordinates from the label
        iou = calculate_iou(pred_box, true_box)
        ious.append(iou)

average_iou = np.mean(ious)
print(f'Average IoU: {average_iou}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 348ms/step
Precision: 0.6628681828757547
Recall: 0.6551921973608721
F1-score: 0.6376964761365077
Average IoU: 0.10389620276665398


C:\Users\LENOVO L460\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
import cv2
import numpy as np

# Load and preprocess the image
def preprocess_image(image_path, input_shape):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255.0  # Normalize pixel values
    return image

# Define the input shape
input_shape = (224, 224, 3)

# Path to the test image
test_image_path = 'C:\\Users\\LENOVO L460\\Downloads\\ipbl\\Capstone-V-Leaukemia Detection\\test\\images\\WBC-Malignant-Pro-303_jpg.rf.64d0bef96ef9527bdd61b0a3978d417a.jpg'


# Preprocess the test image
test_image = preprocess_image(test_image_path, input_shape)

# Reshape the image to match the input shape expected by the model
test_image = np.expand_dims(test_image, axis=0)

# Make predictions
predictions = model.predict(test_image)

# Post-process predictions if needed
# For example, if you're doing object detection, apply NMS or thresholding

# Print the predictions
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[[2.5923727  0.40055847 0.4478162  0.19701983 0.21839455]
  [2.6359968  0.44487423 0.5013212  0.18555976 0.19822629]
  [2.508932   0.5213094  0.51615566 0.16428441 0.1942528 ]
  [2.2431366  0.4409835  0.48734903 0.14500618 0.18796352]
  [1.9592832  0.43140158 0.41518044 0.13126244 0.17454812]]]
